In [29]:
import os
# Dataset location
# Adjust to point to root of dataset
dataset_root = "X:\hirise-map-proj"

In [30]:
# Dataset: 10.5281/zenodo.1048301
from marsvision.utilities import DataUtility
data_loader = DataUtility(os.path.join(dataset_root, "map-proj"))

In [31]:
import pandas as pd

# Apply the SKLearn pipeline--
# Load data, extract features, train a model.
data_loader = DataUtility(os.path.join(dataset_root, "map-proj"))
data_loader.data_reader()
data_loader.data_transformer() 
# The extracted features we're training on should be in data_loader.df
data_df = data_loader.df

# Shaping dataframe to contain features and class codes
lines = pd.read_csv(os.path.join(dataset_root, "labels-map-proj.txt"), 
                    delimiter=" ",  
                    header = None,
                   names=["file_name", "class_code"])
lines = lines.sort_values(by="file_name")

df_labelled = lines.join(data_df.set_index("file_name"), 
                         on="file_name",
                         lsuffix="_left",
                         rsuffix="_right")
df_labelled = df_labelled.drop(labels=["class_code_right", "class"], axis=1)
df_labelled = df_labelled.rename(columns={
    "file_name" : "file_name", 
    "class_code_left" : "class_code"})



In [32]:
df_labelled

,file_name,class_code,0,1,2,3,4,5
873,ESP_011283_2265_RED-0032.jpg,0,84.706379,14424.955996,0.002600,403.657701,102.938210,1758.408455
465,ESP_011289_1950_RED-0006.jpg,6,3.266122,822.193554,-0.001553,45.913871,78.866910,5354.609701
1815,ESP_011289_1950_RED-0007.jpg,6,5.141668,1284.688541,-0.001397,57.873429,60.246386,4771.687281
584,ESP_011289_1950_RED-0009.jpg,6,2.192261,554.220464,-0.000272,49.464534,75.727784,5847.890326
1397,ESP_011289_1950_RED-0013.jpg,6,2.810844,708.864470,0.001979,112.121442,77.699237,5529.686793
...,...,...,...,...,...,...,...,...
1377,TRA_000828_1805_RED-0148.jpg,6,15.058802,3613.226956,0.000252,52.019892,71.678802,5660.806630
513,TRA_000828_1805_RED-0149.jpg,6,11.644220,2833.688185,-0.002193,43.480424,85.822217,5167.247265
2478,TRA_000828_1805_RED-0152.jpg,0,12.139087,2948.109689,-0.001669,16.581029,112.054319,1032.092588
1577,TRA_000828_1805_RED-0154.jpg,0,12.272701,2978.919521,-0.007510,35.245009,82.779309,4322.291647


## Cross Validation - Multi Class Classification With Random Forest

The classes in the Deep Mars dataset are as follows:

0: Other

1: Crater

2: Dark Dune

3: Streak

4: Bright Dune

5: Impact

6: Edge

In [40]:
from sklearn.ensemble import RandomForestClassifier
from marsvision.pipeline import Model
random_forest = RandomForestClassifier(n_estimators = 500)
x = data_loader.images
y = data_loader.labels
random_forest = RandomForestClassifier(n_estimators = 500)
model = Model(random_forest, "sklearn", 
              training_images = x,
              training_labels = y)

c:\users\dpale\desktop\projects\marsvision\marsvision\pipeline\Model.py:57: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.config = yaml.load(yaml_cfg)


In [41]:
multiclass_cv = model.cross_validate(5, "accuracy")

## Cross Validation Results - Per-Class Binary Classification
### "Other" Class

In [43]:
model.training_labels = df_labelled["class_code"].apply(lambda label: 
                                                    1 if label == 0 else 0)
other_cv_results = model.cross_validate_plot("Marsvision Basic Pipeline ROC - 'Other' Class")

ValueError: Found input variables with inconsistent numbers of samples: [3822, 3820]

## Cross Validation Results - Per-Class Binary Classification
### "Crater" Class

In [ ]:
model.training_labels = df_labelled["class_code"].apply(lambda label: 
                                                    1 if label == 1 else 0)
crater_cv_results = model.cross_validate_plot("Marsvision Basic Pipeline ROC - 'Crater' Class")

## Cross Validation Results - Per-Class Binary Classification
### "Dark Dune" Class

In [ ]:
model.training_labels = df_labelled["class_code"].apply(lambda label: 
                                                    1 if label == 2 else 0)
dark_dune_cv_results = model.cross_validate_plot("Marsvision Basic Pipeline ROC - 'Dark Dune' Class")

## Cross Validation Results - Per-Class Binary Classification
### "Streak" Class

In [ ]:
model.training_labels = df_labelled["class_code"].apply(lambda label: 
                                                    1 if label == 3 else 0)
streak_cv_results = model.cross_validate_plot("Marsvision Basic Pipeline ROC - 'Streak' Class")

## Cross Validation Results - Per-Class Binary Classification
### "Bright Dune" Class

In [ ]:
model.training_labels = df_labelled["class_code"].apply(lambda label: 
                                                    1 if label == 4 else 0)
bright_dune_cv_results = model.cross_validate_plot("Marsvision Basic Pipeline ROC - 'Bright Dune' Class")

## Cross Validation Results - Per-Class Binary Classification
### "Edge" Class

In [ ]:
model.training_labels = df_labelled["class_code"].apply(lambda label: 
                                                    1 if label == 6 else 0)
edge_cv_results = model.cross_validate_plot("Marsvision Basic Pipeline ROC - 'Edge' Class")

## Note on the "Impact Class":
There are no samples with this class (impact, class code = 5). Therefore, we cannot train and test for this class.

# Class Frequency
Show histogram of classes to give us an idea of class distributions.

In [ ]:
import seaborn as sns
ax = sns.displot(df_labelled, x="class_code", aspect=1.5)
ax.set(xticks=range(7), title="Deep Mars Class Counts")
ax.set_xticklabels(["Other", "Crater", "Streak", "Dark Dune", "Bright Dune", "Impact", "Edge"])

# Comparing Cross Validation Results Per-Class

These are scores for a Random Forest classifier on each class treated as a binary prediction.

In [ ]:
import numpy as np
results_df = pd.DataFrame(data={
    "class_name": ["Other", "Crater", "Streak", "Dark Dune", "Bright Dune", "Edge"],
    "mean_precision": [
        np.mean(other_cv_results["precisions"]),
        np.mean(crater_cv_results["precisions"]),
        np.mean(streak_cv_results["precisions"]),
        np.mean(dark_dune_cv_results["precisions"]),
        np.mean(bright_dune_cv_results["precisions"]),
        np.mean(edge_cv_results["precisions"])
    ],
    "mean_recall": [
        np.mean(other_cv_results["recalls"]),
        np.mean(crater_cv_results["recalls"]),
        np.mean(streak_cv_results["recalls"]),
        np.mean(dark_dune_cv_results["recalls"]),
        np.mean(bright_dune_cv_results["recalls"]),
        np.mean(edge_cv_results["recalls"])
    ],
    "mean_auc": [
        other_cv_results["mean_auc"],
        crater_cv_results["mean_auc"],
        streak_cv_results["mean_auc"],
        dark_dune_cv_results["mean_auc"],
        bright_dune_cv_results["mean_auc"],
        edge_cv_results["mean_auc"]
    ],
    "mean_acc": [
        other_cv_results["acc_mean"],
        crater_cv_results["acc_mean"],
        streak_cv_results["acc_mean"],
        dark_dune_cv_results["acc_mean"],
        bright_dune_cv_results["acc_mean"],
        edge_cv_results["acc_mean"]]
})



In [ ]:
results_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
ax = sns.barplot(x="class_name", y="mean_precision",
                data=results_df)
ax.set_title("Random Forest | Basic Pipeline Average Precision | 5 Fold CV")

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x="class_name", y="mean_recall",
                data=results_df)
ax.set_title("Basic Pipeline Average Recall | 5 Fold CV")

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x="class_name", y="mean_auc",
                data=results_df)
ax.set_title("Random Forest |Basic Pipeline Average AUC | 5 Fold CV")

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x="class_name", y="mean_acc",
                data=results_df)
ax.set_title("Random Forest | Basic Pipeline Average Accuracy | 5 Fold CV")


The baseline does well identifying the "other" class, which in our dataset means not an object. It did poorly on the Crater and Bright Dune classes, and somewhat better on the Streak and Dark Dune classes as we'd expect. Across the board we have poor recall, which means that this baseline random forest model would not do well in detecting new surface features.

In [ ]:
np.mean(model.cv_results["test_score"])

## Baseline CV Accuracy - All Classes (Mean from 5 Folds): 94.8%

It's worth noting that the seemingly high accuracy is likely due to the imbalance of the "Other" and "Edge" classes.